In [18]:
import xarray as xr 

ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/preprocessed_data.nc')

ds

<xarray.Dataset> Size: 54GB
Dimensions:    (date: 561, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 4kB 1978-01-01 1978-02-01 ... 2024-09-01
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 9kB ...
Data variables: (12/23)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    swvl1      (date, latitude, longitude) float32 2GB ...
    cvh        (date, latitude, longitude) float32 2GB ...
    lai_hv     (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-05T14:05 GRIB to CDM+CF via cfgrib-0.9.1...

Temporal Features

Lets add Seasonal Cycles

In [19]:
# Extract the season from the time coordinate
ds['season'] = ds['date'].dt.season
ds

<xarray.Dataset> Size: 54GB
Dimensions:    (date: 561, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 4kB 1978-01-01 1978-02-01 ... 2024-09-01
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 9kB ...
Data variables: (12/24)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    cvh        (date, latitude, longitude) float32 2GB ...
    lai_hv     (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
    season     (date) <U3 7kB 'DJF' 'DJF' 'MAM' 'MAM' ... 'JJA' 'JJA' 'SON'
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-05T14:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
# Define a dictionary to map season strings to numerical values
season_mapping = {'DJF': 1, 'MAM': 2, 'JJA': 3, 'SON': 4}

# Convert the DataArray to a pandas Series, map the season names, and convert back to a DataArray
ds['season'] = ds['season'].to_series().map(season_mapping).to_xarray()

In [21]:
ds

<xarray.Dataset> Size: 54GB
Dimensions:    (date: 561, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 4kB 1978-01-01 1978-02-01 ... 2024-09-01
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 9kB ...
Data variables: (12/24)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    cvh        (date, latitude, longitude) float32 2GB ...
    lai_hv     (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
    season     (date) int64 4kB 1 1 2 2 2 3 3 3 4 4 4 ... 4 1 1 1 2 2 2 3 3 3 4
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-05T14:05 GRIB to CDM+CF via cfgrib-0.9.1...

Soil Moisture Dynamics

Evapotranspiration Balance: Calculate a moisture balance feature as tp - e (precipitation minus evaporation). This balance can serve as an indicator of moisture availability and stress, especially over rolling windows.

Wind Impact: Combine u10 and v10 (wind components) to create a wind speed variable to assess the drying effect of wind on soil moisture.

Determine Wind Direction: Calculate the wind direction using: Wind speed and direction influence evapotranspiration rates, affecting soil moisture levels.

Temperature Anomalies: Identify deviations from long-term average temperatures to capture unusual thermal conditions impacting soil moisture.

In [25]:
ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/04_feature/features.nc')